# PySpark - Tarea 3
### Practice 1: Task 3: We will continue working with the Heterogeneity Dataset for Human Activity Recognition (HHAR) (full dataset), which contains motion sensor data from smartphones and smartwatches.

In [4]:
# Import libraries
import pyspark
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import*

# Create SparkSession
spark = SparkSession.builder.appName("Merve_Homework").getOrCreate()

## Question1
Generate a Parquet file for each of the original CSV files.

In [5]:
# Create a function to convert the csv file to a parquet file
def write_csv_into_parquet(csv_file):
    """
    This function writes data of a csv file into a parquet file.
    param csv_file: csv file
    return: None
    """
    parquet_file = csv_file.replace(".csv", ".parquet")
    df = spark.read.csv(csv_file, header=False, inferSchema=True)
    df.write.parquet(parquet_file)


write_csv_into_parquet("activity_recognition_exp/Phones_accelerometer.csv")
write_csv_into_parquet("activity_recognition_exp/Phones_gyroscope.csv")
write_csv_into_parquet("activity_recognition_exp/Watch_accelerometer.csv")
write_csv_into_parquet("activity_recognition_exp/Watch_gyroscope.csv")


23/12/02 23:47:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/12/02 23:47:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/12/02 23:47:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/12/02 23:47:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/12/02 23:47:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/12/02 23:47:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/12/02 23:47:09 WARN MemoryManager: Total allocation exceeds 95.

Provide a comparison table showing the sizes of each CSV file and its corresponding Parquet file.

In [14]:
import os

file1_csv = 'activity_recognition_exp/Phones_accelerometer.csv'
file2_csv = 'activity_recognition_exp/Phones_gyroscope.csv'
file3_csv = 'activity_recognition_exp/Watch_accelerometer.csv'
file4_csv = 'activity_recognition_exp/Watch_gyroscope.csv'

file1_pq = 'activity_recognition_exp/Phones_accelerometer.parquet'
file2_pq = 'activity_recognition_exp/Phones_gyroscope.parquet'
file3_pq = 'activity_recognition_exp/Watch_accelerometer.parquet'
file4_pq = 'activity_recognition_exp/Watch_gyroscope.parquet'

csv_files = [file1_csv, file2_csv, file3_csv, file4_csv]
pq_files = [file1_pq, file2_pq, file3_pq, file4_pq]

# Get file size and compare them in Table Format
print("")
print('| {:<30} | {:<10} | {:<15} | {:<20} |'.format(*["File", "Csv (byte)", "Parquet (byte)", "Ratio (Csv/Pq)"]))
print(u'\u2500' * 88)
for file_csv, file_pq in zip(csv_files, pq_files):
    size_csv = os.path.getsize(file_csv)
    size_pq = os.path.getsize(file_pq)
    ratio = size_csv/size_pq

    print('| {:<30} | {:<10} | {:<15} | {:<20} |'.format(file_csv.replace(".csv", "").split("/")[1], size_csv, size_pq, ratio))
print("")


| File                           | Csv (byte) | Parquet (byte)  | Ratio (Csv/Pq)       |
────────────────────────────────────────────────────────────────────────────────────────
| Phones_accelerometer           | 1291856327 | 4096            | 315394.6110839844    |
| Phones_gyroscope               | 1379145657 | 4096            | 336705.4826660156    |
| Watch_accelerometer            | 327168052  | 4096            | 79875.0126953125     |
| Watch_gyroscope                | 308337025  | 4096            | 75277.59399414062    |



## Question2
Measure Execution Time for Different Data Loading Methods

In [17]:
import datetime

# Read parquet files and strore compile time for each file
compile_time_pq = []
for file_pq in pq_files:
    start_time = datetime.datetime.now()
    df = spark.read.parquet(file_pq)
    end_time = datetime.datetime.now()
    compile_time_pq.append((end_time - start_time).total_seconds())


# Read csv files and strore compile time for each file
compile_time_csv = []
for file_csv in csv_files:
    start_time = datetime.datetime.now()
    df = spark.read.csv(file_csv, header=False, inferSchema=True)
    end_time = datetime.datetime.now()
    compile_time_csv.append((end_time - start_time).total_seconds())

# Print compile time for each file in Table Format
print("")
print('| {:<30} | {:<15} | {:<15} | {:<20} |'.format(*["File", "Csv (sec)", "Parquet (sec)", "Ratio (Csv/Pq)"]))
print(u'\u2500' * 93)
for file_csv, file_pq, time_csv, time_pq in zip(csv_files, pq_files, compile_time_csv, compile_time_pq):
    ratio = time_csv/time_pq
    print('| {:<30} | {:<15} | {:<15} | {:<20} |'.format(file_csv.replace(".csv", "").split("/")[1], time_csv, time_pq, ratio))
print("")



| File                           | Csv (sec)       | Parquet (sec)   | Ratio (Csv/Pq)       |
─────────────────────────────────────────────────────────────────────────────────────────────
| Phones_accelerometer           | 3.310691        | 0.045383        | 72.95002533988497    |
| Phones_gyroscope               | 3.446976        | 0.032276        | 106.79687693642335   |
| Watch_accelerometer            | 0.87495         | 0.031636        | 27.656783411303582   |
| Watch_gyroscope                | 0.793566        | 0.031213        | 25.42421426969532    |

